In [22]:
import numpy as np 
import pandas as pd 
import random
import copy

from keras.models import Model
from keras.layers import LSTM, Bidirectional, GlobalMaxPooling1D, Dropout, Embedding
from keras.layers import GlobalAveragePooling1D, concatenate, CuDNNLSTM
from keras.layers.core import SpatialDropout1D
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Input, LSTM, Bidirectional, Conv1D
from gensim.models.keyedvectors import KeyedVectors

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [35]:
sub_cnn = pd.read_csv('0.9869textcnn_submssioncnn-10-fold.csv')
sub_lstm = pd.read_csv('0.98tm_submssionbi-lstm-10-fold.csv')
cv_cnn = pd.read_csv('0.9869textcnn_cv_cnn-10-fold.csv')
cv_lstm = pd.read_csv('0.98tm_cvbi-lstm-10-fold.csv')

In [38]:
cv_cnn.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0001d958c54c6e35,0.025816,0.001058,0.004034,0.001151,0.012525,0.000773
1,00031b1e95af7921,0.003390,0.000075,0.000356,0.000112,0.000834,0.000069
2,00054a5e18b50dd4,0.007660,0.000504,0.001508,0.000395,0.001303,0.000675
3,0005c987bdfc9d4b,0.007647,0.000226,0.002237,0.000210,0.003529,0.000779
4,000c0dfd995809fa,0.001175,0.000213,0.000860,0.000052,0.001429,0.000284


In [39]:
class_list=["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

In [40]:
cv_1 = cv_cnn.reindex(columns=['id']+class_list)
cv_2 = cv_lstm.reindex(columns=['id']+class_list)
sub_1 = sub_cnn.reindex(columns=['id']+class_list)
sub_2 = sub_lstm.reindex(columns=['id']+class_list)

In [41]:
cv_1.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0001d958c54c6e35,0.025816,0.001058,0.004034,0.001151,0.012525,0.000773
1,00031b1e95af7921,0.003390,0.000075,0.000356,0.000112,0.000834,0.000069
2,00054a5e18b50dd4,0.007660,0.000504,0.001508,0.000395,0.001303,0.000675
3,0005c987bdfc9d4b,0.007647,0.000226,0.002237,0.000210,0.003529,0.000779
4,000c0dfd995809fa,0.001175,0.000213,0.000860,0.000052,0.001429,0.000284


In [42]:
cv1_columns = [x+'_cvcnn'for x in class_list]
cv_1.columns = ['id'] + cv1_columns
sub_1_columns = [x+'subcnn'for x in class_list ]
sub_1.columns = ['id'] + sub_1_columns

In [43]:
cv_1.head()

,id,toxic_cvcnn,severe_toxic_cvcnn,obscene_cvcnn,threat_cvcnn,insult_cvcnn,identity_hate_cvcnn
0,0001d958c54c6e35,0.025816,0.001058,0.004034,0.001151,0.012525,0.000773
1,00031b1e95af7921,0.003390,0.000075,0.000356,0.000112,0.000834,0.000069
2,00054a5e18b50dd4,0.007660,0.000504,0.001508,0.000395,0.001303,0.000675
3,0005c987bdfc9d4b,0.007647,0.000226,0.002237,0.000210,0.003529,0.000779
4,000c0dfd995809fa,0.001175,0.000213,0.000860,0.000052,0.001429,0.000284


In [44]:
cv2_columns = [x+'_cvlstm'for x in class_list]
cv_2.columns = ['id'] + cv2_columns
sub_2_columns = [x+'sublstm'for x in class_list ]
sub_2.columns = ['id'] + sub_2_columns

In [59]:
train = pd.read_csv('train_pre2.csv')
test = pd.read_csv('test_pre2.csv')

In [60]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,"explanation,why,the,edits,made,under,my,userna...",0,0,0,0,0,0
1,000103f0d9cfb60f,"d'aww,he,matches,this,background,colour,i,am,s...",0,0,0,0,0,0
2,000113f07ec002fd,"hey,man,i,am,really,not,trying,to,edit,war,it,...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"more,i,can,not,make,any,real,suggestions,on,im...",0,0,0,0,0,0
4,0001d958c54c6e35,"you,sir,are,my,hero,any,chance,you,remember,wh...",0,0,0,0,0,0


In [61]:
train_df = train.merge(cv_1,on=['id'])
train_df = train_df.merge(cv_2,on=['id'])

In [62]:
train_df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,toxic_cvcnn,severe_toxic_cvcnn,obscene_cvcnn,threat_cvcnn,insult_cvcnn,identity_hate_cvcnn,toxic_cvlstm,severe_toxic_cvlstm,obscene_cvlstm,threat_cvlstm,insult_cvlstm,identity_hate_cvlstm
0,0000997932d777bf,"explanation,why,the,edits,made,under,my,userna...",0,0,0,0,0,0,0.002993,0.000229,0.000230,0.000127,0.000342,0.000095,0.000032,2.220235e-08,0.000004,9.837323e-09,3.144886e-07,3.809375e-08
1,000103f0d9cfb60f,"d'aww,he,matches,this,background,colour,i,am,s...",0,0,0,0,0,0,0.001511,0.000129,0.000739,0.000134,0.000206,0.000047,0.000611,2.268425e-05,0.000141,2.601875e-05,5.655173e-05,9.470776e-06
2,000113f07ec002fd,"hey,man,i,am,really,not,trying,to,edit,war,it,...",0,0,0,0,0,0,0.011895,0.000190,0.000454,0.000201,0.001057,0.000200,0.006029,9.674052e-05,0.000782,2.200834e-04,3.416171e-04,3.354928e-05
3,0001b41b1c6bb37e,"more,i,can,not,make,any,real,suggestions,on,im...",0,0,0,0,0,0,0.000109,0.000031,0.000089,0.000009,0.000065,0.000006,0.000011,9.297580e-10,0.000001,1.551985e-08,3.436515e-07,1.982048e-07
4,0001d958c54c6e35,"you,sir,are,my,hero,any,chance,you,remember,wh...",0,0,0,0,0,0,0.025816,0.001058,0.004034,0.001151,0.012525,0.000773,0.006847,3.168830e-05,0.000614,1.038354e-04,8.270378e-04,6.600089e-05


In [63]:
test_df = test.merge(sub_1,on=['id'])
test_df = test_df.merge(sub_2,on=['id'])

In [87]:
test_df.head()

,id,comment_text,toxicsubcnn,severe_toxicsubcnn,obscenesubcnn,threatsubcnn,insultsubcnn,identity_hatesubcnn,toxicsublstm,severe_toxicsublstm,obscenesublstm,threatsublstm,insultsublstm,identity_hatesublstm,cv_id
0,00001cee341fdb12,"yo,bitch,ja,rule,is,more,succesful,then,you'll...",0.999943,0.134518,0.999464,0.008507,0.962728,0.201346,9.976204e-01,4.204700e-01,9.759660e-01,7.548400e-02,9.151845e-01,4.372255e-01,-1
1,0000247867823ef7,"from,rfc,the,title,is,fine,as,it,is,imo",0.000008,0.000019,0.000044,0.000002,0.000089,0.000008,1.042799e-07,4.839774e-10,2.344403e-08,1.458174e-09,9.267739e-09,1.704723e-09,-1
2,00013b17ad220c46,"sources,zawe,ashton,on,lapland,—",0.000320,0.000163,0.000498,0.000017,0.000256,0.000078,9.880099e-06,2.509071e-07,8.679996e-06,5.980463e-07,1.212932e-06,5.879119e-07,-1
3,00017563c3f7919a,"if,you,have,a,look,back,at,the,source,the,info...",0.000024,0.000036,0.000117,0.000053,0.000068,0.000006,8.027424e-06,5.721277e-08,9.731055e-06,1.870044e-06,4.177721e-06,2.636875e-07,-1
4,00017695ad8997eb,"i,do not,anonymously,edit,articles,at,all",0.001430,0.000201,0.000291,0.000049,0.000146,0.000015,9.570464e-05,1.735131e-07,6.355008e-06,9.065370e-07,9.583880e-07,3.244366e-07,-1


In [64]:
train_df['cv_id'] = [random.randint(1,10) for _ in range(len(train_df))]

In [65]:
test_df['cv_id'] = -1

In [66]:
train_df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,toxic_cvcnn,severe_toxic_cvcnn,...,threat_cvcnn,insult_cvcnn,identity_hate_cvcnn,toxic_cvlstm,severe_toxic_cvlstm,obscene_cvlstm,threat_cvlstm,insult_cvlstm,identity_hate_cvlstm,cv_id
0,0000997932d777bf,"explanation,why,the,edits,made,under,my,userna...",0,0,0,0,0,0,0.002993,0.000229,...,0.000127,0.000342,0.000095,0.000032,2.220235e-08,0.000004,9.837323e-09,3.144886e-07,3.809375e-08,2
1,000103f0d9cfb60f,"d'aww,he,matches,this,background,colour,i,am,s...",0,0,0,0,0,0,0.001511,0.000129,...,0.000134,0.000206,0.000047,0.000611,2.268425e-05,0.000141,2.601875e-05,5.655173e-05,9.470776e-06,5
2,000113f07ec002fd,"hey,man,i,am,really,not,trying,to,edit,war,it,...",0,0,0,0,0,0,0.011895,0.000190,...,0.000201,0.001057,0.000200,0.006029,9.674052e-05,0.000782,2.200834e-04,3.416171e-04,3.354928e-05,2
3,0001b41b1c6bb37e,"more,i,can,not,make,any,real,suggestions,on,im...",0,0,0,0,0,0,0.000109,0.000031,...,0.000009,0.000065,0.000006,0.000011,9.297580e-10,0.000001,1.551985e-08,3.436515e-07,1.982048e-07,7
4,0001d958c54c6e35,"you,sir,are,my,hero,any,chance,you,remember,wh...",0,0,0,0,0,0,0.025816,0.001058,...,0.001151,0.012525,0.000773,0.006847,3.168830e-05,0.000614,1.038354e-04,8.270378e-04,6.600089e-05,8


In [74]:
X_train_data = train_df["comment_text"].fillna("fillna").values
y_train_data = train_df[class_list].values
X_test_data = test_df["comment_text"].fillna("fillna").values

In [75]:
max_features = 100000
max_len = 150
embed_size=300

In [76]:
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train_data) + list(X_test_data))

In [79]:
X_train_sequences = tokenizer.texts_to_sequences(X_train_data)
X_test_sequences = tokenizer.texts_to_sequences(X_test_data)

In [80]:
X_train_sequences_pad = sequence.pad_sequences(X_train_sequences, maxlen=max_len)
X_test_sequences_pad = sequence.pad_sequences(X_test_sequences, maxlen=max_len)

In [81]:
word_index = tokenizer.word_index

In [82]:
nb_words = min(max_features, len(word_index))

In [83]:
embedding_vec = 'crawl-300d-2M.vec'
embedding_glove = 'glove.840B.300d.txt'

In [84]:
model = KeyedVectors.load_word2vec_format(embedding_vec, binary=False)

In [88]:
embedding_matrix_fast_text = np.zeros((nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features:
        continue
    embedding_vector = model[word] if word in model else None
    if embedding_vector is not None:
        embedding_matrix_fast_text[i] = embedding_vector
embedding_matrix_fast_text.shape

(100000, 300)

In [104]:
embedding_index_glove = {}
with open(embedding_glove,encoding ='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        cof = np.asarray(values[1:],dtype = 'float32')
        embedding_index_glove[word] = cof

In [105]:
embedding_matrix_glove = np.zeros((nb_words,embed_size))
for word,i in word_index.items():
    if i >= max_features:
        continue
    embedding_vec_glove = embedding_index_glove.get(word)
    if embedding_vec_glove is not None:
        embedding_matrix_glove[i] = embedding_vec_glove

In [106]:
embedding_matrix_glove.shape

(100000, 300)

In [107]:
cv1_x = train_df[cv1_columns].values
cv2_x = train_df[cv2_columns].values

sub1_x = test_df[sub_1_columns].values
sub2_x = test_df[sub_2_columns].values

In [110]:
def get_stacking_model():
    cv1_input = Input(shape=(6,), dtype='float32')
    cv2_input = Input(shape=(6,), dtype='float32')
    merged = concatenate([cv1_input,cv2_input])
    merged = Dense(1000,activation='relu')(merged)
    merged = Dropout(0.1)(merged)
    output = Dense(6, activation='sigmoid')(merged)
    model = Model(inputs=[ cv1_input, cv2_input],
                  outputs=output)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

In [116]:
cv_models=[]
cv_results=[]
cv_scores=[]
Kfold = 10
for i in range(1,Kfold+1):
    idx_train = train_df[train_df['cv_id'] != i].index
    idx_val = train_df[train_df['cv_id'] == i].index
    valid_id = train_df[train_df['cv_id'] == i]['id'].values
    
    cv1_train = cv1_x[idx_train]
    cv2_train = cv2_x[idx_train]

    
    cv1_val = cv1_x[idx_val]
    cv2_val = cv2_x[idx_val]
    
    labels_train = y_train_data[idx_train]
    labels_val = y_train_data[idx_val]
    best = [-1, 0, 0, 0]  
    earlystop = 10
    
    train_ips = [cv1_train, cv2_train]
    val_ips=[cv1_val,cv2_val]
    
    
    model = get_stacking_model()
    
    for epoch in range(100):
        model.fit(train_ips,labels_train,batch_size=256, epochs=1, verbose=1)
        r = model.predict(val_ips ,batch_size=256)
        s = roc_auc_score(labels_val,r)
        print(i,epoch,s)
        if s > best[0]:
            print("epoch " + str(epoch) + " improved from " + str(best[0]) + " to " + str(s))
            best = [s,epoch,copy.copy(model),r]
        if epoch-best[1]>earlystop:
            break

    tpd=pd.DataFrame(columns=[['id']+class_list])
    tpd['id'] = valid_id
    tpd[class_list] = best[-1]
    cv_results.append(tpd)
    cv_models.append(best[2])
    cv_scores.append(best[0])


Epoch 1/1
143473/143473 [==============================] - 3s 20us/step - loss: 0.0866 - acc: 0.9823
1 0 0.9859279861972284
epoch 0 improved from -1 to 0.9859279861972284
Epoch 1/1
143473/143473 [==============================] - 2s 15us/step - loss: 0.0426 - acc: 0.9840
1 1 0.9861219888099858
epoch 1 improved from 0.9859279861972284 to 0.9861219888099858
Epoch 1/1
143473/143473 [==============================] - 2s 15us/step - loss: 0.0420 - acc: 0.9841
1 2 0.9867903826601744
epoch 2 improved from 0.9861219888099858 to 0.9867903826601744
Epoch 1/1
143473/143473 [==============================] - 2s 15us/step - loss: 0.0415 - acc: 0.9842
1 3 0.9868602752178323
epoch 3 improved from 0.9867903826601744 to 0.9868602752178323
Epoch 1/1
143473/143473 [==============================] - 2s 15us/step - loss: 0.0412 - acc: 0.9841
1 4 0.9869774670179147
epoch 4 improved from 0.9868602752178323 to 0.9869774670179147
Epoch 1/1
143473/143473 [==============================] - 2s 15us/step - loss: 0

143473/143473 [==============================] - 2s 15us/step - loss: 0.0395 - acc: 0.9845
1 53 0.9888436771516389
epoch 53 improved from 0.9888125287254966 to 0.9888436771516389
Epoch 1/1
143473/143473 [==============================] - 2s 15us/step - loss: 0.0394 - acc: 0.9845
1 54 0.9887488685114404
Epoch 1/1
143473/143473 [==============================] - 2s 15us/step - loss: 0.0394 - acc: 0.9845
1 55 0.9888255530811804
Epoch 1/1
143473/143473 [==============================] - 2s 15us/step - loss: 0.0394 - acc: 0.9845
1 56 0.9888051686470273
Epoch 1/1
143473/143473 [==============================] - 2s 15us/step - loss: 0.0394 - acc: 0.9845
1 57 0.988506604297621
Epoch 1/1
143473/143473 [==============================] - 2s 15us/step - loss: 0.0394 - acc: 0.9845
1 58 0.9887921274337557
Epoch 1/1
143473/143473 [==============================] - 2s 15us/step - loss: 0.0394 - acc: 0.9846
1 59 0.9884699102712998
Epoch 1/1
143473/143473 [==============================] - 2s 15us/step 

143704/143704 [==============================] - 2s 15us/step - loss: 0.0396 - acc: 0.9845
2 29 0.9875963095030125
Epoch 1/1
143704/143704 [==============================] - 2s 15us/step - loss: 0.0396 - acc: 0.9845
2 30 0.9878111735181531
epoch 30 improved from 0.9877121557341481 to 0.9878111735181531
Epoch 1/1
143704/143704 [==============================] - 2s 15us/step - loss: 0.0395 - acc: 0.9845
2 31 0.9876960266638702
Epoch 1/1
143704/143704 [==============================] - 2s 15us/step - loss: 0.0395 - acc: 0.9845
2 32 0.9876358863998586
Epoch 1/1
143704/143704 [==============================] - 2s 15us/step - loss: 0.0396 - acc: 0.9845
2 33 0.9874608547721633
Epoch 1/1
143704/143704 [==============================] - 2s 15us/step - loss: 0.0395 - acc: 0.9845
2 34 0.9877045297312687
Epoch 1/1
143704/143704 [==============================] - 2s 15us/step - loss: 0.0394 - acc: 0.9844
2 35 0.9877641153888593
Epoch 1/1
143704/143704 [==============================] - 2s 15us/step

143537/143537 [==============================] - 2s 16us/step - loss: 0.0397 - acc: 0.9844
3 24 0.9865149913577672
epoch 24 improved from 0.9862239621519366 to 0.9865149913577672
Epoch 1/1
143537/143537 [==============================] - 2s 15us/step - loss: 0.0396 - acc: 0.9844
3 25 0.9862088287946827
Epoch 1/1
143537/143537 [==============================] - 2s 15us/step - loss: 0.0396 - acc: 0.9845
3 26 0.986521939108887
epoch 26 improved from 0.9865149913577672 to 0.986521939108887
Epoch 1/1
143537/143537 [==============================] - 2s 15us/step - loss: 0.0396 - acc: 0.9845
3 27 0.9866689836526663
epoch 27 improved from 0.986521939108887 to 0.9866689836526663
Epoch 1/1
143537/143537 [==============================] - 2s 15us/step - loss: 0.0396 - acc: 0.9844
3 28 0.9866290617610535
Epoch 1/1
143537/143537 [==============================] - 2s 15us/step - loss: 0.0395 - acc: 0.9845
3 29 0.9860219056617684
Epoch 1/1
143537/143537 [==============================] - 2s 15us/step

143616/143616 [==============================] - 2s 16us/step - loss: 0.0410 - acc: 0.9842
4 5 0.9876540613463929
epoch 5 improved from 0.9875434650117952 to 0.9876540613463929
Epoch 1/1
143616/143616 [==============================] - 2s 15us/step - loss: 0.0409 - acc: 0.9842
4 6 0.9875746262424867
Epoch 1/1
143616/143616 [==============================] - 2s 15us/step - loss: 0.0408 - acc: 0.9841
4 7 0.98753975504357
Epoch 1/1
143616/143616 [==============================] - 2s 15us/step - loss: 0.0406 - acc: 0.9842
4 8 0.9874730099435981
Epoch 1/1
143616/143616 [==============================] - 2s 16us/step - loss: 0.0405 - acc: 0.9842
4 9 0.9878828204561209
epoch 9 improved from 0.9876540613463929 to 0.9878828204561209
Epoch 1/1
143616/143616 [==============================] - 2s 16us/step - loss: 0.0405 - acc: 0.9842
4 10 0.9877968837335412
Epoch 1/1
143616/143616 [==============================] - 2s 15us/step - loss: 0.0404 - acc: 0.9842
4 11 0.9878841754382918
epoch 11 improve

...

143543/143543 [==============================] - 2s 16us/step - loss: 0.0402 - acc: 0.9842
6 10 0.9877255761311495
epoch 10 improved from 0.9871404850014721 to 0.9877255761311495
Epoch 1/1
143543/143543 [==============================] - 2s 16us/step - loss: 0.0402 - acc: 0.9843
6 11 0.9876146849639472
Epoch 1/1
143543/143543 [==============================] - 2s 16us/step - loss: 0.0402 - acc: 0.9843
6 12 0.9875572486440136
Epoch 1/1
143543/143543 [==============================] - 2s 16us/step - loss: 0.0400 - acc: 0.9843
6 13 0.987721789127903
Epoch 1/1
143543/143543 [==============================] - 2s 16us/step - loss: 0.0401 - acc: 0.9843
6 14 0.9875919245091946
Epoch 1/1
143543/143543 [==============================] - 2s 16us/step - loss: 0.0400 - acc: 0.9843
6 15 0.9877105298166787
Epoch 1/1
143543/143543 [==============================] - 2s 16us/step - loss: 0.0400 - acc: 0.9844
6 16 0.9879915647306162
epoch 16 improved from 0.9877255761311495 to 0.9879915647306162
Epoch 1/

143636/143636 [==============================] - 2s 16us/step - loss: 0.0418 - acc: 0.9842
8 2 0.9890406334739086
epoch 2 improved from 0.9885997149461074 to 0.9890406334739086
Epoch 1/1
143636/143636 [==============================] - 2s 16us/step - loss: 0.0414 - acc: 0.9841
8 3 0.989337712897092
epoch 3 improved from 0.9890406334739086 to 0.989337712897092
Epoch 1/1
143636/143636 [==============================] - 2s 16us/step - loss: 0.0411 - acc: 0.9842
8 4 0.9893484951174147
epoch 4 improved from 0.989337712897092 to 0.9893484951174147
Epoch 1/1
143636/143636 [==============================] - 2s 16us/step - loss: 0.0410 - acc: 0.9841
8 5 0.9895287477546288
epoch 5 improved from 0.9893484951174147 to 0.9895287477546288
Epoch 1/1
143636/143636 [==============================] - 2s 16us/step - loss: 0.0407 - acc: 0.9842
8 6 0.9896968884479133
epoch 6 improved from 0.9895287477546288 to 0.9896968884479133
Epoch 1/1
143636/143636 [==============================] - 2s 16us/step - loss

143636/143636 [==============================] - 2s 16us/step - loss: 0.0394 - acc: 0.9845
8 57 0.9907243230418611
Epoch 1/1
143636/143636 [==============================] - 2s 16us/step - loss: 0.0394 - acc: 0.9846
8 58 0.9905734274727213
Epoch 1/1
143636/143636 [==============================] - 2s 16us/step - loss: 0.0394 - acc: 0.9845
8 59 0.9907128452739854
Epoch 1/1
143664/143664 [==============================] - 3s 21us/step - loss: 0.0859 - acc: 0.9825
9 0 0.9841952274677102
epoch 0 improved from -1 to 0.9841952274677102
Epoch 1/1
143664/143664 [==============================] - 2s 16us/step - loss: 0.0424 - acc: 0.9841
9 1 0.984512514566771
epoch 1 improved from 0.9841952274677102 to 0.984512514566771
Epoch 1/1
143664/143664 [==============================] - 2s 16us/step - loss: 0.0417 - acc: 0.9841
9 2 0.9845330093922832
epoch 2 improved from 0.984512514566771 to 0.9845330093922832
Epoch 1/1
143664/143664 [==============================] - 2s 16us/step - loss: 0.0413 - acc:

143594/143594 [==============================] - 2s 16us/step - loss: 0.0401 - acc: 0.9843
10 16 0.988883286620203
epoch 16 improved from 0.9888776981337144 to 0.988883286620203
Epoch 1/1
143594/143594 [==============================] - 2s 16us/step - loss: 0.0401 - acc: 0.9844
10 17 0.9889163637945751
epoch 17 improved from 0.988883286620203 to 0.9889163637945751
Epoch 1/1
143594/143594 [==============================] - 2s 16us/step - loss: 0.0400 - acc: 0.9843
10 18 0.988884976123463
Epoch 1/1
143594/143594 [==============================] - 2s 16us/step - loss: 0.0400 - acc: 0.9844
10 19 0.9889226273917275
epoch 19 improved from 0.9889163637945751 to 0.9889226273917275
Epoch 1/1
143594/143594 [==============================] - 2s 16us/step - loss: 0.0399 - acc: 0.9844
10 20 0.9889008993958169
Epoch 1/1
143594/143594 [==============================] - 2s 16us/step - loss: 0.0399 - acc: 0.9844
10 21 0.9888992543401894
Epoch 1/1
143594/143594 [==============================] - 2s 16us

143594/143594 [==============================] - 2s 16us/step - loss: 0.0393 - acc: 0.9846
10 74 0.989032130038813
Epoch 1/1
143594/143594 [==============================] - 2s 16us/step - loss: 0.0392 - acc: 0.9846
10 75 0.9891312409170281
epoch 75 improved from 0.9890799536102005 to 0.9891312409170281
Epoch 1/1
143594/143594 [==============================] - 2s 16us/step - loss: 0.0392 - acc: 0.9846
10 76 0.9889944693107348
Epoch 1/1
143594/143594 [==============================] - 2s 16us/step - loss: 0.0392 - acc: 0.9846
10 77 0.9890381626071932
Epoch 1/1
143594/143594 [==============================] - 2s 16us/step - loss: 0.0393 - acc: 0.9845
10 78 0.9890698843360292
Epoch 1/1
143594/143594 [==============================] - 2s 16us/step - loss: 0.0393 - acc: 0.9847
10 79 0.9890803594017243
Epoch 1/1
143594/143594 [==============================] - 2s 16us/step - loss: 0.0392 - acc: 0.9846
10 80 0.9889975853504539
Epoch 1/1
143594/143594 [==============================] - 2s 16u

In [117]:
r=[]
avg_val_score = np.average(cv_scores)
print(cv_scores,avg_val_score)
print("prediction begin....")

[0.9889540215867477, 0.9878759928108608, 0.9872794283111722, 0.9885242807562475, 0.989841451610594, 0.9883080848423201, 0.9873505359521674, 0.9907612540604861, 0.9852018803879011, 0.9891312409170281] 0.9883228171235526
prediction begin....


In [118]:
test_ips = [sub1_x, sub2_x]

In [120]:
for i in range(Kfold):
    print("prediction "+ str(i))
    if len(r) == 0:
        r = cv_models[i].predict(test_ips,batch_size=256)
    else:
        r += cv_models[i].predict(test_ips,batch_size=256)

prediction 0
prediction 1
prediction 2
prediction 3
prediction 4
prediction 5
prediction 6
prediction 7
prediction 8
prediction 9


In [121]:
r /= Kfold
index = 'stacking1-'+'10-fold-0'

In [122]:
print("write files...")
pd.concat(cv_results).to_csv("%.4fnn_cv_"% (avg_val_score)+str(index)+".csv",index=False)

sample_submission = pd.read_csv("sample_submission.csv")
sample_submission[class_list] = r

sample_submission.to_csv("%.4f_nn_sub"% (avg_val_score)+ index+".csv",index=False)

write files...
